# 1. Define imports

In [1]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import csv
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

# 2. Load Model 

In [2]:
# Deepfake Detection
df_model = load_model('models/DeepFake_model_ver5_full.keras')

In [3]:
print(df_model)

<Sequential name=sequential_1, built=True>


In [5]:
# Speaker Identification
si_model = load_model('models/speaker_identification_model_ver4_segment.keras')

In [6]:
print(si_model)

<Sequential name=sequential, built=True>


# 3. Evaluation

In [7]:
def reshape_segment(segment_features):
    # Load the scaler
    df_scaler = joblib.load('scalers/df_scaler.pkl') # Deepfake Detection
    si_scaler = joblib.load('scalers/vr_scaler.pkl') # Speaker Identification
    
    # Scale the features using the pre-fitted scaler
    # reshaped = df_scaler.transform([segment_features])  # Deepfake Detection
    reshaped = si_scaler.transform([segment_features])  # Speaker Identification
    
    # Reshape to match the model's input shape (num_features, 1, 1)
    reshaped = reshaped.reshape(1, reshaped.shape[1], 1, 1)  # Shape: (1, num_features, 1, 1)
    print(reshaped.shape)
    return reshaped

In [36]:
def evaluate_segments(segment_data):
    confidence_scores = []  # Store the scores for each segment
    # iterate each row
    for i in range(0, segment_data.shape[0]):
        # Reshape the segment features to match the model's input
        segment_row = segment_data[i]
        
        reshaped_segment = reshape_segment(segment_row)
        
        # Predict the confidence score using the model
        # predictions = df_model.predict(reshaped_segment) # Deepfake Detection
        predictions = si_model.predict(reshaped_segment) # Speaker Identification
    
        confidence = predictions[0][0]
        
        # Store the confidence score
        confidence_scores.append(confidence)

    return confidence_scores


# 4. Load single data

In [33]:
# Deepfake Detection test
# segment_data = pd.read_csv('result\deepfake_detection\\tests\human_t1.csv') # human
# segment_data = pd.read_csv('result\deepfake_detection\\tests\human_t2.csv') # human
# segment_data = pd.read_csv('result\deepfake_detection\\tests\synthetic_t1.csv') # synthetic
segment_data = pd.read_csv('result\deepfake_detection\\tests\synthetic_t2.csv') # synthetic

# Speaker Identification test
# segment_data = pd.read_csv('result\deepfake_detection\\tests\human_t2.csv') # same
# segment_data = pd.read_csv('result\\voice_recognition\\test_full\\vr_full_segment_same.csv') # same

"""

Output: 
Deepfake Detection = 1 Human : 0 AI
Speaker Identification = 1 Speaker : 0 Not Speaker

"""

print(segment_data)
segment_data = segment_data.drop(columns=['file_name']).values
confidence_scores = evaluate_segments(segment_data)

              file_name  mfcc_feature1  mfcc_feature2  mfcc_feature3  \
0  spanish86_cloned.mp3     -231.05008     166.200260      -3.426227   
1  spanish86_cloned.mp3     -245.75290     125.308860     -12.399857   
2  spanish86_cloned.mp3     -271.77322     168.020300     -12.131187   
3  spanish86_cloned.mp3     -263.93347     162.262500      -2.011666   
4  spanish86_cloned.mp3     -279.63278     103.469950      -9.841961   
5  spanish86_cloned.mp3     -244.67416     121.952675     -10.998677   
6  spanish86_cloned.mp3     -246.63835     142.049670      -1.761654   
7  spanish86_cloned.mp3     -217.77368     117.173830     -14.918809   

   mfcc_feature4  mfcc_feature5  mfcc_feature6  mfcc_feature7  mfcc_feature8  \
0      15.043535     -19.883448       1.742834      -9.552059     -17.607178   
1      45.833702      -9.979475       3.966356     -20.320597      -9.616380   
2       4.511300       0.471838      -3.502837     -13.341041     -18.732885   
3      22.318602      -0.007543

In [35]:
print(confidence_scores)

def Average(confidence_scores): 
    return sum(confidence_scores) / len(confidence_scores) 

overall = Average(confidence_scores)
print(overall)

[0.9994083, 0.4171299, 0.99999446, 0.9999987, 1.2969033e-07, 1.0, 1.0, 0.9848684]
0.8001749863910881
